# Federated RX Image Classifier
This example demonstrates how a simple image classifier written in PyTorch can be trained using federated learning with PySyft.



## Install required dependencies

In [1]:
!pip install torch
!pip install torchvision
!pip install syft

In [0]:
#Import required libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import syft as sy  # PySyft library

In [0]:
# Epochs for training/validation
epochs = 5

In [0]:
# Define args object to store model hyperparameters

class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 1
        self.log_interval = 200
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

## Instantiate workers using Hooks

In [0]:
# Instantiate hooks and create workers

hook = sy.TorchHook(torch)

radiology_A = sy.VirtualWorker(hook, id="Radiology_A")
radiology_B = sy.VirtualWorker(hook, id="Radiology_B")

In [0]:
# Load dataset from torchvision module distribute to workers using PySyft's Federated Data loader

def load_data():
        

    federated_train_loader = sy.FederatedDataLoader(
    datasets.CIFAR10('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ]))
    .federate((radiology_A, radiology_B)), # distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

    test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)
    
    return federated_train_loader,test_loader

## Training function

In [0]:
class RXClassifier(nn.Module):
    def __init__(self):
        super(RXClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [0]:
def train(args, model, device, train_loader, optimizer, epoch):

    model.train()

    for batch_idx, (data, target) in enumerate(federated_train_loader): # now it is a distributed dataset

        model.send(data.location) # send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # get the model back

        if batch_idx % args.log_interval == 0:
            loss = loss.get() # get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size,
                100. * batch_idx / len(train_loader), loss.item()))

## Test function

In [0]:
def test(args, model, device, test_loader):

    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
      
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Workflow execution

This step is only used for demo purposes, since edge devices would have their own data in a real-world application.

In [12]:
# Load federated training data and test data
federated_train_loader,test_loader=load_data()

Extracting ../data/cifar-10-python.tar.gz to ../data



In [13]:
# Create Neural Network model instance
model = RXClassifier().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr)

# Train Neural network and validate with test set after completion of training every epoch
for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "RX-Classifier.pt")

Train Epoch: 1 [0/50048 (0%)]	Loss: 2.310682
Train Epoch: 1 [12800/50048 (26%)]	Loss: 2.301998
Train Epoch: 1 [25600/50048 (51%)]	Loss: 2.293859
Train Epoch: 1 [38400/50048 (77%)]	Loss: 2.298970

Test set: Average loss: 2.2804, Accuracy: 1528/10000 (15%)

Train Epoch: 2 [0/50048 (0%)]	Loss: 2.285189
Train Epoch: 2 [12800/50048 (26%)]	Loss: 2.214281
Train Epoch: 2 [25600/50048 (51%)]	Loss: 2.174301
Train Epoch: 2 [38400/50048 (77%)]	Loss: 1.991645

Test set: Average loss: 1.9646, Accuracy: 2947/10000 (29%)

Train Epoch: 3 [0/50048 (0%)]	Loss: 1.924952
Train Epoch: 3 [12800/50048 (26%)]	Loss: 1.949415
Train Epoch: 3 [25600/50048 (51%)]	Loss: 1.895669
Train Epoch: 3 [38400/50048 (77%)]	Loss: 1.820386

Test set: Average loss: 1.7785, Accuracy: 3498/10000 (35%)

Train Epoch: 4 [0/50048 (0%)]	Loss: 1.611385
Train Epoch: 4 [12800/50048 (26%)]	Loss: 1.611142
Train Epoch: 4 [25600/50048 (51%)]	Loss: 1.711248
Train Epoch: 4 [38400/50048 (77%)]	Loss: 1.616658

Test set: Average loss: 1.6442, Accu